# Notebook Imports

In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
from sklearn.model_selection import RandomizedSearchCV

import statistics
from statistics import mode

import joblib

import time

# Writing a Pick-Suggesting Function

## First Pick - Phase 1 Function

In [2]:
df_reshaped = pd.read_pickle('./saved_files/data_final.pickle')

In [3]:
# filter out battles where player on the left lost and picked second
first_pick_list =  df_reshaped[(df_reshaped['first_pick'] == 1) & (df_reshaped['is_win'] == 1)]
first_pick_list




,left_pick_order_1,left_pick_order_2,left_pick_order_3,left_pick_order_4,left_pick_order_5,left_hero_code_1,left_hero_code_2,left_hero_code_3,left_hero_code_4,left_hero_code_5,...,right_job_cd_3,right_job_cd_4,right_job_cd_5,right_postban,first_pick,is_win,left_preban_1,left_preban_2,right_preban_1,right_preban_2
0,1,2,3,4,5,Laia,Blood Moon Haste,Moon Bunny Dominiel,Melissa,Sez,...,assassin,warrior,assassin,Ambitious Tywin,1,1,Zio,New Moon Luna,Sea Phantom Politis,New Moon Luna
2,1,2,3,4,5,Sea Phantom Politis,Blood Moon Haste,Aria,Eternal Wanderer Ludwig,Requiem Roana,...,assassin,mage,assassin,Cidd,1,1,Zio,New Moon Luna,Ran,New Moon Luna
3,1,2,3,4,5,Laia,Blood Moon Haste,Moon Bunny Dominiel,Melissa,Roy Mustang,...,warrior,assassin,mage,Celine,1,1,Zio,New Moon Luna,Sea Phantom Politis,Peira
4,1,2,3,4,5,Laia,Ambitious Tywin,Moon Bunny Dominiel,Roy Mustang,Melissa,...,knight,warrior,mage,Destina,1,1,New Moon Luna,Zio,Nahkwol,Sea Phantom Politis
7,1,2,3,4,5,Laia,Blood Moon Haste,Moon Bunny Dominiel,Roy Mustang,Melissa,...,assassin,knight,assassin,Ran,1,1,New Moon Luna,Zio,Sea Phantom Politis,New Moon Luna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9930,1,2,3,4,5,Laia,Abyssal Yufine,Crimson Armin,Urban Shadow Choux,Moon Bunny Dominiel,...,ranger,manauser,assassin,Desert Jewel Basar,1,1,Jenua,Sea Phantom Politis,New Moon Luna,Death Dealer Ray
9932,1,2,3,4,5,Laia,Nahkwol,Crimson Armin,Celine,ae-KARINA,...,ranger,assassin,mage,Urban Shadow Choux,1,1,Jenua,Sea Phantom Politis,Sea Phantom Politis,New Moon Luna
9933,1,2,3,4,5,Laia,Dragon Bride Senya,Crimson Armin,ae-KARINA,Solitaria of the Snow,...,warrior,knight,manauser,Midnight Gala Lilias,1,1,Jenua,Sea Phantom Politis,Laia,Abyssal Yufine
9938,1,2,3,4,5,Laia,Abyssal Yufine,Crimson Armin,Nahkwol,Elvira,...,warrior,manauser,manauser,New Moon Luna,1,1,Jenua,Sea Phantom Politis,Helen,Melany


In [4]:
first_pick_list['left_hero_code_1'].value_counts()

left_hero_code_1
Laia                       715
New Moon Luna              473
Dragon Bride Senya         358
Blood Moon Haste           344
Death Dealer Ray           330
Sea Phantom Politis        293
Jenua                      113
Ambitious Tywin             99
Zio                         80
Conqueror Lilias            69
Ran                         28
Celine                      24
Cidd                        15
Politis                     13
Lilias                      13
Abyssal Yufine              10
Peira                        8
Briar Witch Iseria           8
Urban Shadow Choux           6
Navy Captain Landy           5
Angel of Light Angelica      5
Unbound Knight Arowell       4
Crimson Armin                4
Sylvan Sage Vivian           3
Watcher Schuri               3
Belian                       2
Albedo                       2
Rimuru                       1
ae-KARINA                    1
Destina                      1
Abigail                      1
Lua                   

In [5]:
df_reshaped['left_hero_code_1'].value_counts()

left_hero_code_1
Laia                       2283
New Moon Luna              1446
Blood Moon Haste           1214
Dragon Bride Senya         1169
Death Dealer Ray           1130
Sea Phantom Politis         943
Jenua                       327
Ambitious Tywin             309
Zio                         251
Conqueror Lilias            217
Ran                         115
Celine                       94
Cidd                         70
Lilias                       54
Abyssal Yufine               45
Politis                      44
Nahkwol                      22
Briar Witch Iseria           20
Peira                        17
Urban Shadow Choux           17
Albedo                       16
Watcher Schuri               13
Crimson Armin                12
Unbound Knight Arowell       11
Angel of Light Angelica      10
Navy Captain Landy            9
Sylvan Sage Vivian            7
Belian                        3
Schuri                        3
Last Rider Krau               2
Wanderer Silk          

In [6]:
# Calculate winrate per hero picked first
winrate = pd.DataFrame((first_pick_list['left_hero_code_1'].value_counts(), df_reshaped['left_hero_code_1'].value_counts()), index = ['win', 'ttl']).T
winrate.dropna(inplace = True)
winrate['win_rate'] = winrate['win']/winrate['ttl']
winrate.index

Index(['Laia', 'New Moon Luna', 'Dragon Bride Senya', 'Blood Moon Haste',
       'Death Dealer Ray', 'Sea Phantom Politis', 'Jenua', 'Ambitious Tywin',
       'Zio', 'Conqueror Lilias', 'Ran', 'Celine', 'Cidd', 'Politis', 'Lilias',
       'Abyssal Yufine', 'Peira', 'Briar Witch Iseria', 'Urban Shadow Choux',
       'Navy Captain Landy', 'Angel of Light Angelica',
       'Unbound Knight Arowell', 'Crimson Armin', 'Sylvan Sage Vivian',
       'Watcher Schuri', 'Belian', 'Albedo', 'Rimuru', 'ae-KARINA', 'Destina',
       'Abigail', 'Lua'],
      dtype='object', name='left_hero_code_1')

In [7]:
# Write a function suggesting a pick based on the most frequent picks and highest win rates
def fp_1(prebans = []):
    """ 
    First pick suggestion
    
    Parameters:
    - List of heroes that were prebanned at the begining of a battle
    
    Returns:
    - str: a hero name based on the most frequently
    picked heroes with the highest win rate
    - str: list of prebans
    """
    mask = winrate.index.isin(prebans)
    hero_list_fp1 = winrate[~mask]
    hero_name = hero_list_fp1.sort_values(['win', 'win_rate'], ascending = False).index[0]
    print('First Pick:', hero_name)
    return hero_name, prebans

In [8]:
# Test the function
fp_1_var = fp_1(['Laia', 'Death Dealer Ray', 'Death Dealer Ray', 'New Moon Luna'])

First Pick: Dragon Bride Senya


<a id='fp23_func_link'></a>
## First Pick - Phase 2 Function

In [9]:
# Adding variables for the function
# defining features and targets
# select features
fp_2p_features = df_reshaped[['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3',
                              'right_pick_order_1', 'right_pick_order_2', 
                              'left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3',
                              'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3',
                              'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3',
                              'right_attack_damage_1', 'right_attack_damage_2',
                              'right_receive_damage_1', 'right_receive_damage_2',
                              'right_kill_count_1', 'right_kill_count_2', 
                              'left_attribute_cd_1', 
                              'left_hero_code_1', 
                              'left_job_cd_1', 
                              'right_hero_code_1', 'right_hero_code_2',
                              'right_attribute_cd_1', 'right_attribute_cd_2' ,
                              'right_job_cd_1', 'right_job_cd_2',
                              'left_preban_1', 'left_preban_2', 
                              'right_preban_1', 'right_preban_2', 'first_pick', 'is_win']]

fp_2p_target = df_reshaped[['left_hero_code_2', 'left_hero_code_3']]

X_train_fp_2p, X_test_fp_2p, y_train_fp_2p, y_test_fp_2p = train_test_split(fp_2p_features, fp_2p_target, test_size= 0.2, shuffle = True, random_state= 890)

# Scaling 
# Define numeric features
fp_2p_numeric = ['left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 
                 'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3',
                 'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3',
                 'right_attack_damage_1', 'right_attack_damage_2', 
                 'right_receive_damage_1', 'right_receive_damage_2',
                 'right_kill_count_1', 'right_kill_count_2']


# Scale numeric features
fp_2p_scaler = StandardScaler()
fp_2p_scaler.fit(X_train_fp_2p[fp_2p_numeric])
X_train_fp_2p_trans = pd.DataFrame()
X_test_fp_2p_trans = pd.DataFrame()

X_train_fp_2p_trans[fp_2p_numeric] = fp_2p_scaler.transform(X_train_fp_2p[fp_2p_numeric])
X_test_fp_2p_trans[fp_2p_numeric] = fp_2p_scaler.transform(X_test_fp_2p[fp_2p_numeric])

# Encoding the categorical features
# Initializing an ordinal encoder for categorical variables
fp_2p_enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

# categorical labels for the 1st iter
fp_2p_categorical = ['left_attribute_cd_1', 
                     'left_hero_code_1',
                     'left_job_cd_1',
                     'right_hero_code_1', 'right_hero_code_2',
                     'right_attribute_cd_1', 'right_attribute_cd_2' ,
                     'right_job_cd_1', 'right_job_cd_2', 
                     'left_preban_1', 'left_preban_2', 
                     'right_preban_1', 'right_preban_2']

X_train_fp_2p_trans[fp_2p_categorical] = fp_2p_enc.fit_transform(X_train_fp_2p[fp_2p_categorical].to_numpy())
X_test_fp_2p_trans[fp_2p_categorical] = fp_2p_enc.transform(X_test_fp_2p[fp_2p_categorical].to_numpy())
X_train_fp_2p_trans.index
X_test_fp_2p_trans.index
X_train_fp_2p = X_train_fp_2p.reset_index(drop=True)
X_test_fp_2p = X_test_fp_2p.reset_index(drop=True)

# add features left out of scaling
l0f_fp_2p = ['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3',
                    'right_pick_order_1', 'right_pick_order_2', 
                   'first_pick', 'is_win']

X_train_fp_2p_trans[l0f_fp_2p] = X_train_fp_2p[l0f_fp_2p]
X_test_fp_2p_trans[l0f_fp_2p] = X_test_fp_2p[l0f_fp_2p]

# Loading df with hero codes and names
dict_fp_2p = pd.read_pickle('./saved_files/dict_fp_2p.pickle')

# Loading the model
classifier_fp_2p = joblib.load('./saved_files/classifier_fp_2p.joblib')

In [10]:
fp_2p_features

,left_pick_order_1,left_pick_order_2,left_pick_order_3,right_pick_order_1,right_pick_order_2,left_attack_damage_1,left_attack_damage_2,left_attack_damage_3,left_receive_damage_1,left_receive_damage_2,...,right_attribute_cd_1,right_attribute_cd_2,right_job_cd_1,right_job_cd_2,left_preban_1,left_preban_2,right_preban_1,right_preban_2,first_pick,is_win
0,1,2,3,1,2,0,1267,0,0,29297,...,dark,light,manauser,knight,Zio,New Moon Luna,Sea Phantom Politis,New Moon Luna,1,1
1,1,2,3,1,2,3112,0,53383,0,0,...,light,light,knight,warrior,Zio,New Moon Luna,Blood Moon Haste,Jenua,0,1
2,1,2,3,1,2,0,4084,6434,0,93232,...,light,fire,knight,assassin,Zio,New Moon Luna,Ran,New Moon Luna,1,1
3,1,2,3,1,2,2043,16469,1228,51842,19373,...,dark,light,manauser,knight,Zio,New Moon Luna,Sea Phantom Politis,Peira,1,1
4,1,2,3,1,2,0,0,0,0,0,...,dark,wind,manauser,manauser,New Moon Luna,Zio,Nahkwol,Sea Phantom Politis,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9941,1,2,3,1,2,23619,0,4764,74779,0,...,dark,light,warrior,knight,Sea Phantom Politis,Jenua,Ran,New Moon Luna,0,2
9942,1,2,3,1,2,3235,0,0,17022,75373,...,light,wind,knight,assassin,Sea Phantom Politis,Jenua,Sea Phantom Politis,Jenua,0,2
9943,1,2,3,1,2,497,3219,12001,34720,46092,...,dark,wind,manauser,assassin,Sea Phantom Politis,Jenua,Jenua,New Moon Luna,1,1
9944,1,2,3,1,2,11488,38100,0,85987,126078,...,dark,fire,manauser,mage,Sea Phantom Politis,Jenua,Navy Captain Landy,Belian,0,2


In [11]:
# Get mean value of encoded numeric features 
battle_stats = fp_2p_features[fp_2p_numeric].astype(float).mean().values.reshape(1,15)
battle_stats.shape

(1, 15)

In [12]:
battle_stats[0][0]

8358.296401132227

In [13]:
def fp_2_3(enemy_pick_1,
           enemy_pick_2, 
           left_preban_1 = fp_1_var[1][0],
           left_preban_2 = fp_1_var[1][1],
           right_preban_1 = fp_1_var[1][2],
           right_preban_2 = fp_1_var[1][3],
           my_fp1 = fp_1_var[0]):
    
    """
    Second and third picks suggestion

    - Parameters: 1st and 2nd heroes picked by the opponent
    
    Returns:
    - array: hero names
    """
        
    test_features_2_3 = pd.DataFrame(columns = X_train_fp_2p_trans.columns)
    # configure stats
    #print(test_features_2_3.loc[0, 'left_preban_1'])
    #print(dict_fp_2p[dict_fp_2p['left_preban_1_orig'] == left_preban_1]['left_preban_1'].values[0])
    test_features_2_3.loc[0, 'left_preban_1'] = dict_fp_2p[dict_fp_2p['left_preban_1_orig'] == left_preban_1]['left_preban_1'].values[0] # left_preban_1 
    test_features_2_3.loc[0, 'left_preban_2'] = dict_fp_2p[dict_fp_2p['left_preban_2_orig'] == left_preban_2]['left_preban_2'].values[0] # left_preban_2 
    test_features_2_3.loc[0, 'right_preban_1'] = dict_fp_2p[dict_fp_2p['right_preban_1_orig'] == right_preban_1]['right_preban_1'].values[0] # right_preban_1 
    test_features_2_3.loc[0, 'right_preban_2'] = dict_fp_2p[dict_fp_2p['right_preban_2_orig'] == right_preban_2]['right_preban_2'].values[0] # right_preban_2 
    test_features_2_3.loc[0, 'right_hero_code_1'] = dict_fp_2p[dict_fp_2p['right_hero_code_1_orig'] == enemy_pick_1]['right_hero_code_1'].values[0] # right_hero_code_1 
    test_features_2_3.loc[0, 'right_hero_code_2'] = dict_fp_2p[dict_fp_2p['right_hero_code_2_orig'] == enemy_pick_2]['right_hero_code_2'].values[0] # right_hero_code_2
    
    
    #print(dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == my_fp1]['left_attribute_cd_1'].values)
    test_features_2_3.loc[0, 'left_attribute_cd_1'] = dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == my_fp1]['left_attribute_cd_1'].values[0] # left_attribute_cd_1
    # test_features_2_3.loc[0, 'left_attribute_cd_2'] = dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == hero_names_2_3[0]]['left_attribute_cd_2'].values[0] # left_attribute_cd_2
    test_features_2_3.loc[0, 'left_hero_code_1'] = dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == my_fp1]['left_hero_code_1'].values[0] # left_hero_code_1
    # test_features_2_3.loc[0, 'left_attribute_cd_3'] = dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == hero_names_2_3[1]]['left_attribute_cd_3'].values[0] # left_attribute_cd_3
    test_features_2_3.loc[0, 'left_job_cd_1'] = dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == my_fp1]['left_job_cd_1'].values[0] # left_job_cd_1
    # test_features_2_3.loc[0, 'left_job_cd_2'] = dict_fp_2p[dict_fp_2p['left_hero_code_1_orig'] == hero_names_2_3[0]]['left_job_cd_2'].values[0].values[0] # left_job_cd_2
    # test_features_2_3.loc[0, 'left_job_cd_3'] = dict_fp_2p[dict_fp_2p['left_hero_code_3_orig'] == hero_names_2_3[1]]['left_job_cd_3'].values[0] # left_job_cd_3
    test_features_2_3.loc[0, 'right_attribute_cd_1'] = dict_fp_2p[dict_fp_2p['right_hero_code_1_orig'] == enemy_pick_1]['right_attribute_cd_1'].values[0] # right_attribute_cd_1
    test_features_2_3.loc[0, 'right_attribute_cd_2'] = dict_fp_2p[dict_fp_2p['right_hero_code_2_orig'] == enemy_pick_2]['right_attribute_cd_2'].values[0] # right_attribute_cd_2
    test_features_2_3.loc[0, 'right_job_cd_1'] = dict_fp_2p[dict_fp_2p['right_hero_code_1_orig'] == enemy_pick_1]['right_job_cd_1'].values[0] # right_job_cd_1
    test_features_2_3.loc[0, 'right_job_cd_2'] = dict_fp_2p[dict_fp_2p['right_hero_code_2_orig'] == enemy_pick_2]['right_job_cd_2'].values[0] # right_job_cd_2

    
    # supply mean numeric values as default encoded numeric features
    battle_stats = fp_2p_features[fp_2p_numeric].astype(float).mean().values.reshape(1,15)
    for i in range(len(test_features_2_3[fp_2p_numeric].columns)):
        # print(battle_stats[i])
        # print(test_features_2_3[fp_2p_numeric[i]])
        test_features_2_3[fp_2p_numeric[i]] = battle_stats[0][i]
    
    # supply unencoded numeric values as default numeric features
    test_features_2_3['left_pick_order_1'] = 1 
    test_features_2_3['left_pick_order_2'] = 2
    test_features_2_3['left_pick_order_3'] = 3
    test_features_2_3['right_pick_order_1'] = 1
    test_features_2_3['right_pick_order_2'] = 2
    test_features_2_3['first_pick'] = 1
    test_features_2_3['is_win'] = 1
    
              
    # make prediction
    # print(len(test_features_2_3.columns))
    # print(test_features_2_3.head())
    hero_names_2_3 = classifier_fp_2p.predict(test_features_2_3)
    print('2nd & 3rd pick:', hero_names_2_3)
    return hero_names_2_3, enemy_pick_1, enemy_pick_2

In [14]:
# Test the function
fp_2_3(enemy_pick_1 = 'Blood Moon Haste',
       enemy_pick_2 = 'Ocean Breeze Luluca')


2nd & 3rd pick: [['Albedo' 'Abyssal Yufine']]


(array([['Albedo', 'Abyssal Yufine']], dtype=object),
 'Blood Moon Haste',
 'Ocean Breeze Luluca')

In [15]:
fp_2_3_var = fp_2_3(enemy_pick_1 = 'Blood Moon Haste', enemy_pick_2 = 'Ocean Breeze Luluca')

2nd & 3rd pick: [['Albedo' 'Abyssal Yufine']]


In [16]:
type(fp_2_3_var[0][0])
fp_2_3_var[0][0][0]

'Albedo'

## First Pick - Phase 3 Function

In [17]:
# Adding variables for the function

# Defining features and targets

# Select features
fp_3p_features = df_reshaped[['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4', 'left_pick_order_5',
                        'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 'right_pick_order_4',
                        'left_attack_damage_1', 'left_attack_damage_2','left_attack_damage_3',
                        'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3',
                        'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3',
                        'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3', 'right_attack_damage_4',
                        'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3', 'right_receive_damage_4',
                        'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3', 'right_kill_count_4',
                        'left_hero_code_1', 'left_hero_code_2', 'left_hero_code_3',
                        'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3',
                        'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3', 
                        'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3', 'right_hero_code_4',
                        'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3', 'right_attribute_cd_4',
                        'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3', 'right_job_cd_4',
                        'left_preban_1', 'left_preban_2', 'right_preban_1', 'right_preban_2', 'first_pick', 'is_win']]

fp_3p_target = df_reshaped[['left_hero_code_4', 'left_hero_code_5']]

X_train_fp_3p, X_test_fp_3p, y_train_fp_3p, y_test_fp_3p = train_test_split(fp_3p_features, fp_3p_target, test_size= 0.2, shuffle = True, random_state= 890)

# Scaling 
# Define numeric features
fp_3p_numeric = ['left_attack_damage_1', 'left_attack_damage_2', 'left_attack_damage_3', 
                 'left_receive_damage_1', 'left_receive_damage_2', 'left_receive_damage_3', 
                 'left_kill_count_1', 'left_kill_count_2', 'left_kill_count_3', 
                 'right_attack_damage_1', 'right_attack_damage_2', 'right_attack_damage_3', 'right_attack_damage_4',
                 'right_receive_damage_1', 'right_receive_damage_2', 'right_receive_damage_3', 'right_receive_damage_4',
                 'right_kill_count_1', 'right_kill_count_2', 'right_kill_count_3', 'right_kill_count_4']


# Scale numeric features
fp_3p_scaler = StandardScaler()
fp_3p_scaler.fit(X_train_fp_3p[fp_3p_numeric])
X_train_fp_3p_trans = pd.DataFrame()
X_test_fp_3p_trans = pd.DataFrame()
X_train_fp_3p_trans[fp_3p_numeric] = fp_3p_scaler.transform(X_train_fp_3p[fp_3p_numeric])
X_test_fp_3p_trans[fp_3p_numeric] = fp_3p_scaler.transform(X_test_fp_3p[fp_3p_numeric])

# Encoding the categorical features
# Initializing an ordinal encoder for categorical variables
fp_3p_enc = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

# categorical labels
fp_3p_categorical = ['left_hero_code_1', 'left_hero_code_2', 'left_hero_code_3',
                     'left_attribute_cd_1', 'left_attribute_cd_2', 'left_attribute_cd_3', 
                     'left_job_cd_1', 'left_job_cd_2', 'left_job_cd_3',
                     'right_hero_code_1', 'right_hero_code_2', 'right_hero_code_3', 'right_hero_code_4',
                     'right_attribute_cd_1', 'right_attribute_cd_2', 'right_attribute_cd_3', 'right_attribute_cd_4',
                     'right_job_cd_1', 'right_job_cd_2', 'right_job_cd_3', 'right_job_cd_4',
                     'left_preban_1', 'left_preban_2',
                     'right_preban_1', 'right_preban_2']

X_train_fp_3p_trans[fp_3p_categorical] = fp_3p_enc.fit_transform(X_train_fp_3p[fp_3p_categorical].to_numpy())
X_test_fp_3p_trans[fp_3p_categorical] = fp_3p_enc.transform(X_test_fp_3p[fp_3p_categorical].to_numpy())

X_train_fp_3p = X_train_fp_3p.reset_index(drop=True)
X_test_fp_3p = X_test_fp_3p.reset_index(drop=True)

# add features left out of scaling
l0f_fp_3p = ['left_pick_order_1', 'left_pick_order_2', 'left_pick_order_3', 'left_pick_order_4', 'left_pick_order_5',
             'right_pick_order_1', 'right_pick_order_2', 'right_pick_order_3', 'right_pick_order_4', 
             'first_pick', 'is_win']

X_train_fp_3p_trans[l0f_fp_3p] = X_train_fp_3p[l0f_fp_3p]
X_test_fp_3p_trans[l0f_fp_3p] = X_test_fp_3p[l0f_fp_3p]

# Loading df with hero codes and names
dict_fp_3p = pd.read_pickle('./saved_files/dict_fp_3p.pickle')

# Loading the model
classifier_fp_3p = joblib.load('./saved_files/classifier_fp_3p.joblib')

In [18]:
def fp_4_5(enemy_pick_1,
           enemy_pick_2,
           enemy_pick_3,
           enemy_pick_4,
           left_preban_1 = fp_1_var[1][0],
           left_preban_2 = fp_1_var[1][1],
           right_preban_1 = fp_1_var[1][2],
           right_preban_2 = fp_1_var[1][3],
           my_fp1 = fp_1_var[0],
           my_fp2 = fp_2_3_var[0][0][0],
           my_fp3 = fp_2_3_var[0][0][1]):
    
    """
    Second and third picks suggestion

    - Parameters: 3rd and 4th heroes picked by the opponent
    
    Returns:
    List of hero names
    """
    
    test_features_4_5 = pd.DataFrame(columns = X_train_fp_3p_trans.columns)
    # configure stats
    # prebans
    test_features_4_5.loc[0, 'left_preban_1'] = dict_fp_3p[dict_fp_3p['left_preban_1_orig'] == left_preban_1]['left_preban_1'].values[0] # left_preban_1
    test_features_4_5.loc[0, 'left_preban_2'] = dict_fp_3p[dict_fp_3p['left_preban_2_orig'] == left_preban_2]['left_preban_2'].values[0] # left_preban_2
    test_features_4_5.loc[0, 'right_preban_1'] = dict_fp_3p[dict_fp_3p['right_preban_1_orig'] == right_preban_1]['right_preban_1'].values[0] # right_preban_1
    test_features_4_5.loc[0, 'right_preban_2'] = dict_fp_3p[dict_fp_3p['right_preban_2_orig'] == right_preban_2]['right_preban_2'].values[0] # right_preban_2
    
    # right side picks
    test_features_4_5.loc[0, 'right_attribute_cd_1'] = dict_fp_3p[dict_fp_3p['right_hero_code_1_orig'] == enemy_pick_1]['right_attribute_cd_1'].values[0] # right_attribute_cd_1
    test_features_4_5.loc[0, 'right_attribute_cd_2'] = dict_fp_3p[dict_fp_3p['right_hero_code_2_orig'] == enemy_pick_2]['right_attribute_cd_2'].values[0] # right_attribute_cd_2
    test_features_4_5.loc[0, 'right_attribute_cd_3'] = dict_fp_3p[dict_fp_3p['right_hero_code_3_orig'] == enemy_pick_3]['right_attribute_cd_3'].values[0] # right_attribute_cd_3
    test_features_4_5.loc[0, 'right_attribute_cd_4'] = dict_fp_3p[dict_fp_3p['right_hero_code_4_orig'] == enemy_pick_4]['right_attribute_cd_4'].values[0] # right_attribute_cd_4
    test_features_4_5.loc[0, 'right_job_cd_1'] = dict_fp_3p[dict_fp_3p['right_hero_code_1_orig'] == enemy_pick_1]['right_job_cd_1'].values[0] # right_job_cd_1
    test_features_4_5.loc[0, 'right_job_cd_2'] = dict_fp_3p[dict_fp_3p['right_hero_code_2_orig'] == enemy_pick_2]['right_job_cd_2'].values[0] # right_job_cd_2
    test_features_4_5.loc[0, 'right_job_cd_3'] = dict_fp_3p[dict_fp_3p['right_hero_code_3_orig'] == enemy_pick_3]['right_job_cd_3'].values[0] # right_job_cd_3
    test_features_4_5.loc[0, 'right_job_cd_4'] = dict_fp_3p[dict_fp_3p['right_hero_code_4_orig'] == enemy_pick_4]['right_job_cd_4'].values[0] # right_job_cd_4    
    test_features_4_5.loc[0, 'right_hero_code_1'] = dict_fp_3p[dict_fp_3p['right_hero_code_1_orig'] == enemy_pick_1]['right_hero_code_1'].values[0] # right_hero_code_1
    test_features_4_5.loc[0, 'right_hero_code_2'] = dict_fp_3p[dict_fp_3p['right_hero_code_2_orig'] == enemy_pick_2]['right_hero_code_2'].values[0] # right_hero_code_2
    test_features_4_5.loc[0, 'right_hero_code_3'] = dict_fp_3p[dict_fp_3p['right_hero_code_3_orig'] == enemy_pick_3]['right_hero_code_3'].values[0] # right_hero_code_3
    test_features_4_5.loc[0, 'right_hero_code_4'] = dict_fp_3p[dict_fp_3p['right_hero_code_4_orig'] == enemy_pick_4]['right_hero_code_4'].values[0] # right_hero_code_4
    
    # left side picks
    test_features_4_5.loc[0, 'left_attribute_cd_1'] = dict_fp_3p[dict_fp_3p['left_hero_code_1_orig'] == my_fp1]['left_attribute_cd_1'].values[0] # left_attribute_cd_1
    test_features_4_5.loc[0, 'left_attribute_cd_2'] = dict_fp_3p[dict_fp_3p['left_hero_code_2_orig'] == my_fp2]['left_attribute_cd_2'].values[0] # left_attribute_cd_2
    test_features_4_5.loc[0, 'left_attribute_cd_3'] = dict_fp_3p[dict_fp_3p['left_hero_code_3_orig'] == my_fp3]['left_attribute_cd_3'].values[0] # left_attribute_cd_3
    test_features_4_5.loc[0, 'left_job_cd_1'] = dict_fp_3p[dict_fp_3p['left_hero_code_1_orig'] == my_fp1]['left_job_cd_1'].values[0] # left_job_cd_1
    test_features_4_5.loc[0, 'left_job_cd_2'] = dict_fp_3p[dict_fp_3p['left_hero_code_2_orig'] == my_fp2]['left_job_cd_2'].values[0] # left_job_cd_2
    test_features_4_5.loc[0, 'left_job_cd_3'] = dict_fp_3p[dict_fp_3p['left_hero_code_3_orig'] == my_fp3]['left_job_cd_3'].values[0] # left_job_cd_3
    test_features_4_5.loc[0, 'left_hero_code_1'] = dict_fp_3p[dict_fp_3p['left_hero_code_1_orig'] == my_fp1]['left_hero_code_1'].values[0] # left_hero_code_1
    test_features_4_5.loc[0, 'left_hero_code_2'] = dict_fp_3p[dict_fp_3p['left_hero_code_2_orig'] == my_fp2]['left_hero_code_2'].values[0] # left_hero_code_2
    test_features_4_5.loc[0, 'left_hero_code_3'] = dict_fp_3p[dict_fp_3p['left_hero_code_3_orig'] == my_fp3]['left_hero_code_3'].values[0] # left_hero_code_3
        
    
    # supply mean numeric values as default encoded numeric features
    battle_stats = fp_2p_features[fp_2p_numeric].astype(float).mean().values.reshape(1,15)
    for i in range(len(test_features_4_5[fp_2p_numeric].columns)):
        test_features_4_5[fp_2p_numeric[i]] = battle_stats[0][i]
    
    # supply unencoded numeric values as default numeric features
    test_features_4_5['left_pick_order_1'] = 1 
    test_features_4_5['left_pick_order_2'] = 2
    test_features_4_5['left_pick_order_3'] = 3
    test_features_4_5['left_pick_order_4'] = 4
    test_features_4_5['left_pick_order_5'] = 5
    test_features_4_5['right_pick_order_1'] = 1
    test_features_4_5['right_pick_order_2'] = 2
    test_features_4_5['right_pick_order_3'] = 3
    test_features_4_5['right_pick_order_4'] = 4
    test_features_4_5['first_pick'] = 1
    test_features_4_5['is_win'] = 1
    
              
    # make prediction
    hero_names_3_4 = classifier_fp_3p.predict(test_features_4_5)
    print('4th & 5th pick:', hero_names_3_4)
    return hero_names_3_4

In [19]:
# Test the function
fp_4_5(enemy_pick_1 = 'Blood Moon Haste', 
       enemy_pick_2 = 'Ocean Breeze Luluca',
       enemy_pick_3 = 'Savior Adin',
       enemy_pick_4 = 'Navy Captain Landy')

2nd & 3rd pick: [['Nahkwol' 'Midnight Gala Lilias']]


array([['Nahkwol', 'Midnight Gala Lilias']], dtype=object)

In [32]:
print(dict_fp_3p[['left_hero_code_1_orig', 'left_hero_code_2_orig', 'left_hero_code_3_orig']].to_string())

        left_hero_code_1_orig     left_hero_code_2_orig     left_hero_code_3_orig
0         Sea Phantom Politis          Blood Moon Haste          Death Dealer Ray
1               New Moon Luna                       Ran   Eternal Wanderer Ludwig
2                        Laia                    Celine       Moon Bunny Dominiel
3            Death Dealer Ray           Ambitious Tywin       Moon Bunny Dominiel
4                        Laia        Dragon Bride Senya                    Albedo
5            Death Dealer Ray           Ambitious Tywin        Navy Captain Landy
6         Sea Phantom Politis                     Jenua                   Nahkwol
7                       Jenua                   Politis                      Amid
8         Sea Phantom Politis                     Jenua        Silvertide Christy
9               New Moon Luna                       Zio   Eternal Wanderer Ludwig
10              New Moon Luna    Unbound Knight Arowell              Specimen Sez
11              

## Testing Functions

In [41]:
fp_1_var = fp_1(['Ambitious Tywin', 'Navy Captain Landy', 'Flan', 'Pavel'])

First Pick: Laia


In [42]:
fp_2_3_var = fp_2_3(enemy_pick_1 = 'New Moon Luna', enemy_pick_2 = 'Death Dealer Ray')

2nd & 3rd pick: [['Blood Moon Haste' 'Belian']]


In [43]:
fp_4_5(enemy_pick_1 = 'New Moon Luna', 
       enemy_pick_2 = 'Death Dealer Ray',
       enemy_pick_3 = 'Specimen Sez',
       enemy_pick_4 = 'Celine')

2nd & 3rd pick: [['Nahkwol' 'Nahkwol']]


array([['Nahkwol', 'Nahkwol']], dtype=object)